# Agentic RAG

In [1]:
# %pip install -r "../requirements.txt"

In [2]:
from dotenv import load_dotenv

load_dotenv()

from knowledge_base import retriever
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.tools import tool, ToolRuntime
from langchain.agents import create_agent, AgentState
from langgraph.types import Command
from langchain_core.documents.base import Document
from langchain.messages import ToolMessage
from pydantic import BaseModel


## Tool definition

In [3]:
class CustomAgentState(AgentState):
    user_preferences: dict
    user_name: str | None

class CustomContext(BaseModel):
    user_id: str

In [4]:
@tool
def greet_user(runtime: ToolRuntime) -> str | Command:
    """Use to greet the user"""
    if user:=runtime.state.get('user_name'):
        return f'Hello, {user}'
    return Command(
        update={
            "messages": [
                ToolMessage(
                    "Please call the 'update_user_info' tool it will get and update the user's name.",
                    tool_call_id=runtime.tool_call_id
                )
            ]
        }
    )


In [5]:
@tool
def update_user_info(runtime: ToolRuntime) -> Command:
    """Use to update user info when user name is unknown"""
    print('called')
    user_id: str = runtime.context.user_id
    user_name = 'Balaji' if 'blj' in user_id else 'Guest'
    return Command(
        update={
            "user_name": user_name,
            "messages": [
                ToolMessage(
                    "Successfully looked up user information",
                    tool_call_id=runtime.tool_call_id
                )
            ]
        }
    )
        

In [6]:
@tool
def get_to_know_the_user(runtime: ToolRuntime) -> str:
    """Use to get to know user and opening a conversation with the user"""
    user_preferences: dict = runtime.state['user_preferences']
    return f'User prefers the following: {user_preferences}'

In [7]:
@tool
def get_context(query: str) -> tuple[str, list[Document]]:
    """Retrieve information to help answer a query."""
    retrieved_docs = retriever.invoke(query)
    serialized = "\n\n".join(
        (f"Source: {doc.metadata}\nContent: {doc.page_content}")
        for doc in retrieved_docs
    )
    return serialized, retrieved_docs

## Agent Prompt

In [26]:
prompt = """
You are a helpful Clerical assistant. You have access to the following tool:
`greet_user`:Use to greet the user
`update_user_info`: to update user info when user name is unknown
`get_to_know_the_user`: This is used to get to know the user preferences and assist the user accordingly.
`get_context`: This can be used if you need to find any file information for the user's question. 
It takes in a search parameter 'q' as input and this is used to perform vector search in the vector database having file information.
The tool returns a tuple containing information text having content, file name and document metadata information.

< Answer >
Always greet the user. If name is found, always specify in the start of the greeting and if no name, use the `update_user_info` TOOL to find the proper name, use Guest to greet.
Always remember get to know the user for their preferences using `get_to_know_the_user` tool. Preferences might be anything like answer format, response tone, response length, etc.
Respond to the user question in a friendly manner. If its a general question and if you know the answer, feel free to answer and insist the user that you can help with clerical information.
In case you have used the `get_context` tool and it was helpful, let the user know you have searched for the files and found the most appropriate information in the file 'foo'
</Answer>
"""

### Memory saver for Short term memory

In [9]:
from langgraph.checkpoint.memory import InMemorySaver
cp = InMemorySaver()

## Agent creation

In [27]:
agent = create_agent(
    model=ChatGoogleGenerativeAI(model="gemini-2.5-flash-lite"),
    tools=[greet_user, update_user_info, get_to_know_the_user, get_context],
    system_prompt=prompt,
    state_schema=CustomAgentState,
    context_schema=CustomContext,
    checkpointer=cp
 )

In [16]:
def ask_ai(q, thread_id, user_id, user_name, **preferences):
    print(preferences)
    res = agent.invoke(
    {
        "messages": [
            {"role": "user", "content": q}
        ],
        "user_preferences": {
            **preferences
        },
        'user_name': user_name
    },
    context=CustomContext(user_id=user_id),
    config={
        'configurable': {'thread_id': thread_id}
    }
    )

    for msg in res['messages']:
        msg.pretty_print()

In [32]:
ask_ai(
    q='hi, get me the customers for bluray', 
    thread_id=1222202, 
    user_id='ablj',
    user_name='BB',
    answer_format='detailed list'
)

{'answer_format': 'detailed list'}
================================ Human Message =================================

hi, get me the customers for bluray
================================== Ai Message ==================================
Tool Calls:
  get_context (6141091a-6517-4692-94c0-791a05eade17)
 Call ID: 6141091a-6517-4692-94c0-791a05eade17
  Args:
    query: customers for bluray
================================= Tool Message =================================
Name: get_context

("Source: {'page': 0, 'title': 'Document', 'source': 'docs/invoice_Aaron Hawkins_40100.pdf', 'creator': 'wkhtmltopdf 0.12.6', 'producer': 'Qt 4.8.7', 'page_label': '1', 'total_pages': 1, 'creationdate': '2023-06-05T19:21:16+00:00'}\nContent: INVOICE\n# 40100\nSuperStore\nBill To\n:\nAaron Hawkins\nShip To\n:\n10035, New York City,\nNew York, United\nStates\nDec 30 2012\nFirst Class\n$112.53\nDate\n:\nShip Mode\n:\nBalance Due\n:\nItem\nQuantity\nRate\nAmount\nVerbatim 25 GB 6x Blu-ray Single Layer Recordable 

In [29]:
ask_ai('hey, who purchased bluray', user_id='blj', user_name=None, thread_id=11, answer_format='list')

{'answer_format': 'list'}
================================ Human Message =================================

hey, who purchased bluray
================================== Ai Message ==================================
Tool Calls:
  greet_user (ea2c7581-4998-4e12-ae19-e0fc59fae407)
 Call ID: ea2c7581-4998-4e12-ae19-e0fc59fae407
  Args:
  get_to_know_the_user (9285586c-9f77-48a0-ab3e-308abeea338d)
 Call ID: 9285586c-9f77-48a0-ab3e-308abeea338d
  Args:
================================= Tool Message =================================
Name: greet_user

Please call the 'update_user_info' tool it will get and update the user's name.
================================= Tool Message =================================
Name: get_to_know_the_user

User prefers the following: {'answer_format': 'list'}
================================== Ai Message ==================================

Hello Guest, I hope you are doing well. I'm not sure who purchased bluray, but I can help you with other clerical tasks.
What